In [1]:
import smtplib
import json
from email.message import EmailMessage
from datetime import datetime

In [5]:
# Your Bible Reading Sections
bible_sections = {
    "Genesis-Deuteronomy": [("Genesis", 50), ("Exodus", 40), ("Leviticus", 27), ("Numbers", 36), ("Deuteronomy", 34)],
    "Joshua-2 Kings": [("Joshua", 24), ("Judges", 21), ("Ruth", 4), ("1 Samuel", 31), ("2 Samuel", 24), ("1 Kings", 22), ("2 Kings", 25)],
    "1 Chronicles-Job": [("1 Chronicles", 29), ("2 Chronicles", 36), ("Ezra", 10), ("Nehemiah", 13), ("Esther", 10), ("Job", 42)],
    "Psalm": [("Psalm", 150)],
    "Proverbs-Song of Solomon": [("Proverbs", 31), ("Ecclesiastes", 12), ("Song of Solomon", 8)],
    "Isaiah-Ezekiel": [("Isaiah", 66), ("Jeremiah", 52), ("Lamentations", 5), ("Ezekiel", 48)],
    "Daniel-Malachi": [("Daniel", 12), ("Hosea", 14), ("Joel", 3), ("Amos", 9), ("Obadiah", 1), ("Jonah", 4), ("Micah", 7), ("Nahum", 3), ("Habakkuk", 3), ("Zephaniah", 3), ("Haggai", 2), ("Zechariah", 14), ("Malachi", 4)],
    "Matthew-John": [("Matthew", 28), ("Mark", 16), ("Luke", 24), ("John", 21)],
    "Acts": [("Acts", 28)],
    "Romans-Colossians": [("Romans", 16), ("1 Corinthians", 16), ("2 Corinthians", 13), ("Galatians", 6), ("Ephesians", 6), ("Philippians", 4), ("Colossians", 4)],
    "1 Thessalonians-2 Peter": [("1 Thessalonians", 5), ("2 Thessalonians", 3), ("1 Timothy", 6), ("2 Timothy", 4), ("Titus", 3), ("Philemon", 1), ("Hebrews", 13), ("James", 5), ("1 Peter", 5), ("2 Peter", 3)],
    "1 John-Revelation": [("1 John", 5), ("2 John", 1), ("3 John", 1), ("Jude", 1), ("Revelation", 22)]
}


progress_file = 'reading_progress.json'

# Load or Initialize Progress
def load_or_initialize_progress():
    try:
        with open(progress_file, 'r') as file:
            return json.load(file)
    except FileNotFoundError:
        return {section: (section_data[0][0], 1) for section, section_data in bible_sections.items()}

# Update Progress
def update_progress(progress):
    with open(progress_file, 'w') as file:
        json.dump(progress, file)

# Get the next chapter in a section
def get_next_chapter(section, current_book, current_chapter):
    books = bible_sections[section]
    for i, (book, chapters) in enumerate(books):
        if book == current_book:
            if current_chapter < chapters:
                return current_book, current_chapter + 1
            elif i + 1 < len(books):
                return books[i + 1][0], 1
            else:
                return books[0][0], 1
    return None  # Fallback, should not occur

# Send Email
def send_email(reading_plan, sender_email, receiver_email, password):
    message = EmailMessage()
    message.set_content("\n".join([f"{section}: {book} Chapter {chapter}" for section, (book, chapter) in reading_plan.items()]))
    message['Subject'] = f"Bible Reading Plan for {datetime.now().strftime('%Y-%m-%d')}"
    message['From'] = sender_email
    message['To'] = receiver_email

    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
        smtp.login(sender_email, password)
        smtp.send_message(message)

# Main Function
def main():
    progress = load_or_initialize_progress()

    next_reading_plan = {}
    for section, (current_book, current_chapter) in progress.items():
        next_book, next_chapter = get_next_chapter(section, current_book, current_chapter)
        next_reading_plan[section] = (next_book, next_chapter)
        progress[section] = (next_book, next_chapter)

    sender_email = ""
    receiver_email = ""
    password = "" # Use an app password if using Gmail
    send_email(next_reading_plan, sender_email, receiver_email, password)

    update_progress(progress)

if __name__ == "__main__":
    main()

